# EEG-2 Feature extraction

In [3]:
import pandas as pd
import numpy as np
import librosa as lib
from scipy.fftpack import fft, fftfreq

In [4]:
df = pd.read_csv(r'data\Epileptic Seizure Recognition.csv')
df.drop('Unnamed', axis=1, inplace=True)

data_nor = df[df['y']!=1].drop('y', axis=1)
data_nor.reset_index(drop=True, inplace = True)
data_epi = df[df['y']==1].drop('y', axis=1)
data_epi.reset_index(drop=True, inplace = True)

## ApEn

In [57]:
def ApEn(U, m=30):
    r = np.std(U)
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])

    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [len([1 for x_j in x if _maxdist(x_i, x_j) <= r]) / (N - m + 1.0) for x_i in x]
        return (N - m + 1.0)**(-1) * sum(np.log(C))

    N = len(U)

    return abs(_phi(m+1) - _phi(m))

## Mean frequency-fft

In [20]:
def mean_freq(y):
    fs = 178
    # signal
    t = np.linspace(0, 1, 178)
    N = len(t)
    # fft
    yf = fft(y)
    xf = fftfreq(N, 1/fs)[:N//2]
    yf_nor = 2.0/N * np.abs(yf[0:N//2]) # normalization
    xf_half = xf[:len(yf_nor)]
    return np.mean(xf_half[yf_nor.argsort()[-2:]])

## Peek

In [11]:
data_nor_N = data_nor.sub(data_nor.mean(axis=1), axis=0)
data_nor_pos = data_nor_N[data_nor_N > 0]
data_nor_neg = data_nor_N[data_nor_N < 0]
data_nor_pospeek = data_nor_N.sub(1.5*data_nor_pos.mean(axis=1), axis=0)
data_nor_negpeek = data_nor_N.sub(1.5*data_nor_neg.mean(axis=1), axis=0)



data_epi_N = data_epi.sub(data_epi.mean(axis=1), axis=0)
data_epi_pos = data_epi_N[data_epi_N > 0]
data_epi_neg = data_epi_N[data_epi_N < 0]
data_epi_pospeek = data_epi_N.sub(1.5*data_epi_pos.mean(axis=1), axis=0)
data_epi_negpeek = data_epi_N.sub(1.5*data_epi_neg.mean(axis=1), axis=0)

In [12]:
def nor_SUM_peek(i):
    pospeek = lib.zero_crossings(np.array(data_nor_pospeek.iloc[i]))
    negpeek = lib.zero_crossings(np.array(data_nor_negpeek.iloc[i]))
    return max(np.sum(pospeek==1), np.sum(negpeek==1))

def epi_SUM_peek(i):
    pospeek = lib.zero_crossings(np.array(data_epi_pospeek.iloc[i]))
    negpeek = lib.zero_crossings(np.array(data_epi_negpeek.iloc[i]))
    return max(np.sum(pospeek==1), np.sum(negpeek==1))

## feature

In [ ]:
feature = pd.DataFrame(columns=['ApEn', 'peek', 'RMS', 'mean frequency', 'label'])

for i in np.arange(0, len(data_epi), 1):
    apen = ApEn(np.array(data_epi.iloc[i]).astype(float))
    peek = epi_SUM_peek(i)
    rms = lib.feature.rms(y=np.array(data_epi.iloc[i]).astype(float))[0, 0]
    meanf = mean_freq(np.array(data_epi.iloc[i]).astype(float))
    feature = feature.append(pd.DataFrame({'ApEn':apen, 'peek':peek, 'RMS':rms, 'mean frequency':meanf, 'label':1}, index=[i]))

for i in np.arange(0, len(data_nor), 1):
    apen = ApEn(np.array(data_nor.iloc[i]).astype(float))
    peek = nor_SUM_peek(i)
    rms = lib.feature.rms(y=np.array(data_nor.iloc[i]).astype(float))[0, 0]
    meanf = mean_freq(np.array(data_nor.iloc[i]).astype(float))
    feature = feature.append(pd.DataFrame({'ApEn':apen, 'peek':peek, 'RMS':rms, 'label':0}, index=[i+2300]))
feature.to_csv(r'feature.csv')